In [2]:
from scalpel.cfg import CFGBuilder

def cfg_to_dict(cfg):
    """
    遍历 CFG 和所有子 CFG，生成嵌套的 Python 字典，
    并在每个 block 中记录 start_line 和 end_line。
    """
    visited = set()  # 防止重复访问

    def traverse(block):
        """
        遍历单个 CFG 块并生成节点和边的数据结构
        """
        if block.id in visited:
            return None
        visited.add(block.id)

        # 获取当前块的源代码
        block_label = block.get_source().strip() if not block.is_empty() else "Empty Block"

        # 按行拆分代码
        lines = block_label.split('\n')
        # 去掉包含 `...` 的行
        filtered_lines = [line for line in lines if '...' not in line]
        # 将过滤后的行重新合并为块标签
        block_label = '\n'.join(filtered_lines).strip()

        # ============ 核心逻辑：根据 statements 来获取行号 ============ 
        if block.statements:
            # 如果可以保证 block.statements 是有序的
            start_line = block.statements[0].lineno
            # 如果没有 end_lineno，就尝试用最后一条语句的 lineno
            end_line = block.statements[-1].end_lineno or block.statements[-1].lineno
        else:
            start_line = None
            end_line = None
        # ==========================================================

        block_dict = {
            "id": block.id,
            "label": block_label,
            "start_line": start_line,
            "end_line": end_line,
            "successors": []
        }

        # 遍历后继节点
        for exit_ in block.exits:
            successor = traverse(exit_.target)
            if successor:
                block_dict["successors"].append(successor)

        return block_dict

    def process_cfg(cfg, prefix="Main"):
        """
        处理当前 CFG，包括其子 CFG（函数和类），并返回嵌套字典
        """
        cfg_dict = {
            "name": cfg.name,
            "type": "CFG",
            "blocks": []
        }

        # 处理当前 CFG 的入口块
        if cfg.entryblock:
            entry_block = traverse(cfg.entryblock)
            if entry_block:
                cfg_dict["blocks"].append(entry_block)

        # 递归处理函数的子 CFG
        cfg_dict["functions"] = []
        for func_name, func_cfg in cfg.functioncfgs.items():
            func_dict = process_cfg(func_cfg, prefix=f"{prefix}_func_{func_name}")
            cfg_dict["functions"].append(func_dict)

        # 递归处理类的子 CFG
        cfg_dict["classes"] = []
        for class_name, class_cfg in cfg.class_cfgs.items():
            class_dict = process_cfg(class_cfg, prefix=f"{prefix}_class_{class_name}")
            cfg_dict["classes"].append(class_dict)

        return cfg_dict

    return process_cfg(cfg)

import json
import os

for i, file in enumerate(os.listdir("../../dataset/python")):
    with open("../../dataset/python/" + file, "r") as f:
        source_code = f.read()

    # 构建控制流图
    cfg = CFGBuilder().build_from_src(file, source_code)

    # 将 CFG 转换为字典数据结构
    cfg_dict = cfg_to_dict(cfg)

    # 保存到文件
    with open("../../dataset/python_cfg/" + file.replace(".py", ".json"), "w") as f:
        json.dump(cfg_dict, f, indent=2)
